In [196]:
# Based on Kaggle kernel by Scirpus
def convert_to_ffm(df,type,numerics,categories,features):
    currentcode = len(numerics)
    catdict = {}
    catcodes = {}
    # Flagging categorical and numerical fields
    for x in numerics:
         catdict[x] = 0
    for x in categories:
         catdict[x] = 1
    
    nrows = df.shape[0]
    ncolumns = len(features)
    with open(str(type) + "_ffm.txt", "w") as text_file:
# Looping over rows to convert each row to libffm format
        for n,r in enumerate(range(nrows)):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow['Label']))
             # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(catdict.keys()):
                if(catdict[x]==0):
                    datastring = datastring + " "+str(i)+":"+ str(i)+":"+ str(datarow[x])
                else:
            # For a new field appearing in a training example
                    if(x not in catcodes):
                        catcodes[x] = {}
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
            # For already encoded fields
                    elif(datarow[x] not in catcodes[x]):
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
                    code = catcodes[x][datarow[x]]
                    datastring = datastring + " "+str(i)+":"+ str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

In [197]:
import pandas as pd 

In [198]:
df = pd.read_csv('train.csv')

In [199]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [200]:
all_c =['Sex','Embarked']
all_n = list(df.select_dtypes(include=[int,float]).columns[2:])
all_f = all_c+all_n

In [201]:
all_f

['Sex', 'Embarked', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [157]:
#df#

In [202]:
df=df.rename(index=str,columns={'Survived':'Label'})
df.Age=df.Age.fillna(float(int(df.Age.mean())))

In [186]:
#float(int(df.Age.mean()))

In [209]:
convert_to_ffm(df[all_n+['Label']],'Train',all_n,[],all_f)

In [204]:
df[all_n+['Label']].isnull().sum()

Pclass    0
Age       0
SibSp     0
Parch     0
Fare      0
Label     0
dtype: int64

In [205]:
import numpy as np

In [206]:
df_t  = pd.read_csv('test.csv')
df_t['Label']=0
#df_t=df_t.rename(index=str,columns={'Survived':'Label'})

In [149]:
#df_t

In [207]:
convert_to_ffm(df_t[all_n+['Label']],'Test',all_n,[],all_f)

In [210]:
import xlearn as xl

# Training task
ffm_model = xl.create_fm()  # Use field-aware factorization machine
ffm_model.setTrain("Train_ffm.txt")   # Training data
ffm_model.setValidate("Train_ffm.txt")  # Validation data

# param:
#  0. binary classification
#  1. learning rate : 0.2
#  2. regular lambda : 0.002
param = {'task':'binary', # ‘binary’ for classification, ‘reg’ for Regression
         'k':10,           # Size of latent factor 
         'lr':0.1,        # Learning rate for GD
         'lambda':0.0002, # L2 Regularization Parameter
         'metric':'auc',  # Metric for monitoring validation set performance
         'epoch':25       # Maximum number of Epochs
        }
# Train model
ffm_model.fit(param,'model.out')

In [211]:
X.Age = X.Age.fillna(25)

In [212]:
X = df[all_n]
#X=pd.get_dummies(X,columns=X.select_dtypes(include=[object]))
X.Age = X.Age.fillna(25)
y = df.Label

#print df
from sklearn.datasets import dump_svmlight_file
dump_svmlight_file(X,y,'smvlight.dat',zero_based=True,multilabel=False)

In [213]:
ffm_model = xl.create_ffm()
ffm_model.setTrain("Train_ffm.txt")
ffm_model.setValidate("Train_ffm.txt")

param = {'task':'binary', # ‘binary’ for classification, ‘reg’ for Regression
         'k':10,           # Size of latent factor 
         'lr':0.1,        # Learning rate for GD
         'lambda':0.0002, # L2 Regularization Parameter
         'metric':'auc',  # Metric for monitoring validation set performance
         'epoch':25       # Maximum number of Epochs
        }

ffm_model.fit(param, "model.out")

In [214]:
ffm_model.setTest("Test_ffm.txt")
ffm_model.predict('model.out', "output2.txt")

In [215]:
import xlearn as xl

# Training task
fm_model = xl.create_fm()  # Use factorization machine
fm_model.setTrain("Train_ffm.txt")  # Training data
fm_model.setValidate("Train_ffm.txt")
# param:
#  0. Binary classification task
#  1. learning rate: 0.2
#  2. lambda: 0.002
#  3. metric: accuracy
param = {'task':'binary', 'lr':0.2, 'k':5,
         'lambda':0.002, 'metric':'acc'}

# Use cross-validation
fm_model.fit(param,'model.out')

In [95]:
ffm_model.setTest("Test_ffm.txt")
ffm_model.predict('model.out','output1.txt')